In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.5 MB/s eta 0:00:00


# **Spam Detection**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import gradio as gr
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Let's use a larger and more balanced sample dataset for Spam vs. Not Spam classification
# Example data, should be replaced with a proper dataset (e.g., SMS Spam Collection dataset)
data = {
    "text": [
        "Congratulations! You've won a $1000 gift card. Call now!",
        "Hello, I hope you're doing well. Just wanted to check in!",
        "Get free money now! Just click here.",
        "Can we reschedule the meeting to 3pm tomorrow?",
        "Limited time offer! Win a free iPhone today!",
        "Important reminder: Your account has been compromised, act now.",
        "See you tomorrow at the office.",
        "Win a prize in just 5 minutes! Click here to claim it!",
        "Hello! I’m free to meet at 2pm. Does that work for you?",
        "Special offer on electronics today only! Don't miss out!",
        "Exclusive offer just for you, buy one get one free!",
        "Don't miss the sale! Limited time offer!",
        "Meeting scheduled at 10 AM tomorrow.",
        "Important: Action required on your account.",
        "Hello, let's meet at 3 PM today."
    ],
    "label": [1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0]  # 1 = Spam, 0 = Not Spam (Ham)
}

# Convert the dataset into a pandas DataFrame
df = pd.DataFrame(data)

# Extract features using TF-IDF
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(df['text']).toarray()
y = torch.tensor(df['label'])

# Split data into training and test sets (with random_state for reproducibility)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define a Feedforward Neural Network for spam classification
class SpamClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SpamClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x

# Initialize the model, loss function, and optimizer
model = SpamClassifier(input_dim=X_train.shape[1], hidden_dim=16, output_dim=2)  # Spam vs Not Spam (2 classes)
criterion = nn.CrossEntropyLoss()  # Cross-entropy loss for classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Convert training data to torch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = y_train

# Training loop with evaluation and early stopping
num_epochs = 100
best_accuracy = 0

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    # Backward pass and optimization
    loss.backward()
    optimizer.step()

    # Calculate accuracy for the training data
    _, predicted = torch.max(outputs, 1)
    correct = (predicted == y_train_tensor).sum().item()
    accuracy = correct / len(y_train_tensor)

    # Save the model with the best accuracy
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model.state_dict(), "best_spam_model.pth")

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Accuracy: {accuracy:.4f}')

# Load the best model
model.load_state_dict(torch.load("best_spam_model.pth"))

# Function to make predictions using the trained model
def predict_spam(text):
    # Preprocess input text
    text_vector = vectorizer.transform([text]).toarray()
    text_tensor = torch.tensor(text_vector, dtype=torch.float32)

    model.eval()
    with torch.no_grad():
        output = model(text_tensor)
        predicted = torch.argmax(output, dim=1).item()

    return "Spam" if predicted == 1 else "Not Spam"

# Create Gradio interface
interface = gr.Interface(
    fn=predict_spam,  # The function to use for prediction
    inputs="text",  # The input type (text box for user input)
    outputs="text",  # The output type (text label showing prediction)
    title="Spam Detection Model",
    description="Enter a message, and the model will predict whether it's Spam or Not Spam."
)

# Launch Gradio interface
interface.launch()


Epoch [10/100], Loss: 0.7041, Accuracy: 0.4167
Epoch [20/100], Loss: 0.6933, Accuracy: 0.4167
Epoch [30/100], Loss: 0.6813, Accuracy: 0.4167
Epoch [40/100], Loss: 0.6680, Accuracy: 0.4167
Epoch [50/100], Loss: 0.6527, Accuracy: 0.4167
Epoch [60/100], Loss: 0.6353, Accuracy: 0.5000
Epoch [70/100], Loss: 0.6160, Accuracy: 0.7500
Epoch [80/100], Loss: 0.5957, Accuracy: 0.9167
Epoch [90/100], Loss: 0.5746, Accuracy: 0.9167
Epoch [100/100], Loss: 0.5533, Accuracy: 1.0000
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://eb4f3c41db326738d7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spac

# **sentiment analyasis**

In [ ]:
pip install  transformers


In [ ]:
import gradio as gr
from transformers import pipeline

# Load the sentiment analysis pipeline from Hugging Face
sentiment_analyzer = pipeline("sentiment-analysis")

# Define the function for sentiment analysis
def analyze_sentiment(text):
    result = sentiment_analyzer(text)
    return result[0]['label'], result[0]['score']

# Create the Gradio interface
iface = gr.Interface(
    fn=analyze_sentiment,
    inputs=gr.Textbox(label="Enter text for sentiment analysis"),
    outputs=[gr.Label(), gr.Textbox(label="Confidence Score")]
)

# Launch the interface
iface.launch()


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://12ae4ad51de02047b6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
pip install streamlit spacy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.6 MB/s eta 0:00:00


In [ ]:
!pip install streamlit
!pip install pyngrok


In [ ]:
import gradio as gr
import spacy

# Load the spaCy English model
nlp = spacy.load("en_core_web_sm")

# Function to perform POS tagging
def pos_tagging(text):
    # Process the text using spaCy
    doc = nlp(text)
    # Extract words and their corresponding POS tags
    pos_tags = [(token.text, token.pos_) for token in doc]
    return pos_tags

# Create a Gradio interface
iface = gr.Interface(
    fn=pos_tagging,
    inputs=gr.Textbox(label="Enter text for POS tagging"),
    outputs=gr.Dataframe(headers=["Word", "POS Tag"]),
    live=True
)

# Launch the interface
iface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c785af8174ce589e5e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import gradio as gr
from transformers import MarianMTModel, MarianTokenizer

# Mapping of target languages to Hugging Face model names for English to [Target Language]
language_pairs = {
    "French": "Helsinki-NLP/opus-mt-en-fr",
    "Spanish": "Helsinki-NLP/opus-mt-en-es",
    "German": "Helsinki-NLP/opus-mt-en-de",
    "Italian": "Helsinki-NLP/opus-mt-en-it",
    "Portuguese": "Helsinki-NLP/opus-mt-en-pt",
    "Dutch": "Helsinki-NLP/opus-mt-en-nl",
    "Russian": "Helsinki-NLP/opus-mt-en-ru",
    "Arabic": "Helsinki-NLP/opus-mt-en-ar",
    "Hindi": "Helsinki-NLP/opus-mt-en-hi",  # English to Hindi
    "Japanese": "Helsinki-NLP/opus-mt-en-ja",
    "Korean": "Helsinki-NLP/opus-mt-en-ko",
    "Chinese": "Helsinki-NLP/opus-mt-en-zh",
    "Turkish": "Helsinki-NLP/opus-mt-en-tr",
    "Finnish": "Helsinki-NLP/opus-mt-en-fi",
    "Polish": "Helsinki-NLP/opus-mt-en-pl"
}

# Function to load the appropriate model and tokenizer based on the target language
def load_model(target_language):
    model_name = language_pairs.get(target_language)
    model = MarianMTModel.from_pretrained(model_name)
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    return model, tokenizer

# Function to preprocess the input text (optional cleaning)
def preprocess_input(text):
    # Simple replacements for known abbreviations (you can expand this as needed)
    text = text.replace("SRH", "Sunrisers Hyderabad")  # Full form of SRH
    text = text.replace("LSG", "Lucknow Super Giants")  # Full form of LSG
    text = text.replace("MI", "Mumbai Indians")  # Full form of MI
    text = text.replace("PK", "Punjab Kings")  # Full form of PK
    # Add more replacements or fixes if necessary
    return text

# Function to perform translation
def translate(text, target_language):
    text = preprocess_input(text)  # Preprocess the input text
    model, tokenizer = load_model(target_language)
    # Tokenize the input text and translate it
    translated = model.generate(**tokenizer(text, return_tensors="pt", padding=True))
    translated_text = tokenizer.decode(translated[0], skip_special_tokens=True)
    return translated_text

# Create the Gradio interface
iface = gr.Interface(
    fn=translate,
    inputs=[
        gr.Textbox(label="Enter English text for translation"),
        gr.Dropdown(
            label="Choose target language",
            choices=list(language_pairs.keys()),
            value="French"  # Set default value
        )
    ],
    outputs=gr.Textbox(label="Translated text"),
    live=True
)

# Launch the interface
iface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9243aa49ea2e232562.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import gradio as gr
from transformers import pipeline

# Initialize the Hugging Face summarization pipeline
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Function to perform text summarization
def summarize(text):
    summary = summarizer(text, max_length=150, min_length=30, do_sample=False)
    return summary[0]['summary_text']

# Create the Gradio interface
iface = gr.Interface(
    fn=summarize,
    inputs=gr.Textbox(label="Enter Text to Summarize", lines=5),
    outputs=gr.Textbox(label="Summary"),
    live=True
)

# Launch the interface
iface.launch()


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://64076b8cb504c1996b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import gradio as gr
from transformers import pipeline

# Initialize the Hugging Face text generation pipeline
generator = pipeline("text-generation", model="gpt2")

# Function to generate text based on user input
def generate_text(prompt, max_length, num_sequences):
    # Generate text using the GPT-2 model with specified parameters
    generated = generator(prompt, max_length=max_length, num_return_sequences=num_sequences, no_repeat_ngram_size=2)

    # Collect all generated sequences and return as a list of strings
    generated_text = [seq['generated_text'] for seq in generated]
    return "\n\n".join(generated_text)  # Join multiple sequences if needed

# Create the Gradio interface with inputs for prompt, max length, and number of sequences
iface = gr.Interface(
    fn=generate_text,
    inputs=[
        gr.Textbox(label="Enter your prompt", lines=5),
        gr.Slider(label="Max Length", minimum=50, maximum=500, step=10, value=150),
        gr.Slider(label="Number of Sequences", minimum=1, maximum=5, step=1, value=1)
    ],
    outputs=gr.Textbox(label="Generated Text"),
    live=True
)

# Launch the interface
iface.launch()


Device set to use cpu


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://dae677894357af3dfc.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.3/130.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 98.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.7 MB/s eta 0:00:00


In [ ]:
!pip install diffusers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 36.1 MB/s eta 0:00:00


In [ ]:
import gradio as gr
from diffusers import StableDiffusionPipeline
import torch

# Load the pre-trained Stable Diffusion model
# Updated model identifier to runwayml/stable-diffusion-v1-5
pipe = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16)
pipe = pipe.to("cuda")  # Use GPU if available

# Function to generate image from text prompt
def generate_image(prompt):
    # Generate image from prompt
    image = pipe(prompt).images[0]
    return image

# Create Gradio interface for image generation
iface = gr.Interface(
    fn=generate_image,
    inputs=gr.Textbox(label="Enter text prompt for image generation"),
    outputs=gr.Image(label="Generated Image"),
    live=True
)

# Launch the Gradio app
iface.launch()

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
